<a href="https://colab.research.google.com/github/admhan/Airiau/blob/main/AdamHannachi_AnaelErnadote_CarRacing_RL_IASD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proximal Policy Optimitization (PPO)

# Installation et Import

## Installation

In [2]:
!pip install swig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.1 MB/s eta 0:00:00


In [3]:

!pip install stable_baselines3 gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 45.5 MB/s eta 0:00:00


## Import

In [4]:


# =========================
# Stable-Baselines3 callbacks & evaluation
# =========================
from stable_baselines3.common.callbacks import (
    EvalCallback,
    CallbackList,
    CheckpointCallback
)
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import VecVideoRecorder

# =========================
# Utilities
# =========================
import os
import numpy as np
import matplotlib.pyplot as plt
import torch


from stable_baselines3.common.vec_env import VecVideoRecorder
from google.colab import files
import os



Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Fonction

## PPO

In [5]:

from __future__ import annotations

import warnings
from typing import Any, ClassVar, TypeVar

import numpy as np
import torch as th
from torch.nn import functional as F
from gymnasium import spaces

from stable_baselines3.common.buffers import RolloutBuffer
from stable_baselines3.common.on_policy_algorithm import OnPolicyAlgorithm
from stable_baselines3.common.policies import (
    ActorCriticCnnPolicy,
    ActorCriticPolicy,
    BasePolicy,
    MultiInputActorCriticPolicy,
)
from stable_baselines3.common.type_aliases import GymEnv, MaybeCallback, Schedule
from stable_baselines3.common.utils import explained_variance


SelfPPO = TypeVar("SelfPPO", bound="PPO")


class PPO(OnPolicyAlgorithm):
    """
    Proximal Policy Optimization (PPO) — clip variant (SB3-style).
    """

    policy_aliases: ClassVar[dict[str, type[BasePolicy]]] = {
        "MlpPolicy": ActorCriticPolicy,
        "CnnPolicy": ActorCriticCnnPolicy,
        "MultiInputPolicy": MultiInputActorCriticPolicy,
    }

    def __init__(
        self,
        policy: str | type[ActorCriticPolicy],
        env: GymEnv | str,
        learning_rate: float | Schedule = 3e-4,
        n_steps: int = 2048,
        batch_size: int = 64,
        n_epochs: int = 10,
        gamma: float = 0.99,
        gae_lambda: float = 0.95,
        clip_range: float | Schedule = 0.2,
        clip_range_vf: float | Schedule | None = None,
        normalize_advantage: bool = True,
        ent_coef: float = 0.0,
        vf_coef: float = 0.5,
        max_grad_norm: float = 0.5,
        use_sde: bool = False,
        sde_sample_freq: int = -1,
        rollout_buffer_class: type[RolloutBuffer] | None = None,
        rollout_buffer_kwargs: dict[str, Any] | None = None,
        target_kl: float | None = None,
        stats_window_size: int = 100,
        tensorboard_log: str | None = None,
        policy_kwargs: dict[str, Any] | None = None,
        verbose: int = 0,
        seed: int | None = None,
        device: th.device | str = "auto",
        _init_setup_model: bool = True,
    ):
        super().__init__(
            policy,
            env,
            learning_rate=learning_rate,
            n_steps=n_steps,
            gamma=gamma,
            gae_lambda=gae_lambda,
            ent_coef=ent_coef,
            vf_coef=vf_coef,
            max_grad_norm=max_grad_norm,
            use_sde=use_sde,
            sde_sample_freq=sde_sample_freq,
            rollout_buffer_class=rollout_buffer_class,
            rollout_buffer_kwargs=rollout_buffer_kwargs,
            stats_window_size=stats_window_size,
            tensorboard_log=tensorboard_log,
            policy_kwargs=policy_kwargs,
            verbose=verbose,
            device=device,
            seed=seed,
            _init_setup_model=False,
            supported_action_spaces=(
                spaces.Box,
                spaces.Discrete,
                spaces.MultiDiscrete,
                spaces.MultiBinary,
            ),
        )

        if normalize_advantage:
            assert batch_size > 1, "`batch_size` must be > 1 when normalize_advantage=True"

        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.clip_range = clip_range
        self.clip_range_vf = clip_range_vf
        self.normalize_advantage = normalize_advantage
        self.target_kl = target_kl

        if _init_setup_model:
            self._setup_model()

    def _setup_model(self) -> None:
        super()._setup_model()

    def train(self) -> None:
        self.policy.set_training_mode(True)
        self._update_learning_rate(self.policy.optimizer)

        clip_range = (
            self.clip_range(self._current_progress_remaining)
            if callable(self.clip_range)
            else self.clip_range
        )

        clip_range_vf = None
        if self.clip_range_vf is not None:
            clip_range_vf = (
                self.clip_range_vf(self._current_progress_remaining)
                if callable(self.clip_range_vf)
                else self.clip_range_vf
            )

        entropy_losses, pg_losses, value_losses, clip_fractions = [], [], [], []
        approx_kl_divs = []

        for epoch in range(self.n_epochs):
            for rollout_data in self.rollout_buffer.get(self.batch_size):
                actions = rollout_data.actions
                if isinstance(self.action_space, spaces.Discrete):
                    actions = actions.long().flatten()

                values, log_prob, entropy = self.policy.evaluate_actions(
                    rollout_data.observations, actions
                )
                values = values.flatten()

                advantages = rollout_data.advantages
                if self.normalize_advantage and advantages.numel() > 1:
                    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

                ratio = th.exp(log_prob - rollout_data.old_log_prob)

                policy_loss = -th.mean(
                    th.min(
                        advantages * ratio,
                        advantages * th.clamp(ratio, 1 - clip_range, 1 + clip_range),
                    )
                )

                clip_fraction = th.mean((th.abs(ratio - 1) > clip_range).float()).item()

                if clip_range_vf is None:
                    values_pred = values
                else:
                    values_pred = rollout_data.old_values + th.clamp(
                        values - rollout_data.old_values,
                        -clip_range_vf,
                        clip_range_vf,
                    )

                value_loss = F.mse_loss(rollout_data.returns, values_pred)

                entropy_loss = -th.mean(entropy) if entropy is not None else th.mean(log_prob)

                loss = policy_loss + self.ent_coef * entropy_loss + self.vf_coef * value_loss

                with th.no_grad():
                    approx_kl = th.mean(
                        (th.exp(log_prob - rollout_data.old_log_prob) - 1)
                        - (log_prob - rollout_data.old_log_prob)
                    ).cpu().item()
                    approx_kl_divs.append(approx_kl)

                if self.target_kl is not None and approx_kl > 1.5 * self.target_kl:
                    break

                self.policy.optimizer.zero_grad()
                loss.backward()
                th.nn.utils.clip_grad_norm_(self.policy.parameters(), self.max_grad_norm)
                self.policy.optimizer.step()

                pg_losses.append(policy_loss.item())
                value_losses.append(value_loss.item())
                entropy_losses.append(entropy_loss.item())
                clip_fractions.append(clip_fraction)

        explained_var = explained_variance(
            self.rollout_buffer.values.flatten(),
            self.rollout_buffer.returns.flatten(),
        )

        self.logger.record("train/policy_gradient_loss", np.mean(pg_losses))
        self.logger.record("train/value_loss", np.mean(value_losses))
        self.logger.record("train/entropy_loss", np.mean(entropy_losses))
        self.logger.record("train/approx_kl", np.mean(approx_kl_divs))
        self.logger.record("train/clip_fraction", np.mean(clip_fractions))
        self.logger.record("train/explained_variance", explained_var)
        self.logger.record("train/n_updates", self._n_updates, exclude="tensorboard")

    def learn(
        self: SelfPPO,
        total_timesteps: int,
        callback: MaybeCallback = None,
        log_interval: int = 1,
        tb_log_name: str = "PPO",
        reset_num_timesteps: bool = True,
        progress_bar: bool = False,
    ) -> SelfPPO:
        return super().learn(
            total_timesteps=total_timesteps,
            callback=callback,
            log_interval=log_interval,
            tb_log_name=tb_log_name,
            reset_num_timesteps=reset_num_timesteps,
            progress_bar=progress_bar,
        )

## Callbacks

In [6]:
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np
import os

class BestModelCallback(BaseCallback):
    def __init__(self, model_ref, save_path, verbose=1):
        super().__init__(verbose)
        self.model_ref = model_ref
        self.save_path = save_path
        self.best_reward = -np.inf

        os.makedirs(os.path.dirname(save_path), exist_ok=True)

    def _on_step(self) -> bool:
        infos = self.locals.get("infos", [])

        for info in infos:
            if "episode" in info:
                r = info["episode"]["r"]

                if r > self.best_reward:
                    self.best_reward = r
                    self.model_ref.save(self.save_path)

                    if self.verbose:
                        print(f"🏆 New BEST model saved | Reward = {r:.2f}")

        return True


In [7]:
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np

class EpisodeRewardCallback(BaseCallback):
    def __init__(self, verbose=1):
        super().__init__(verbose)
        self.episode_rewards = []
        self.current_reward = 0
        self.episode_count = 0

    def _on_step(self) -> bool:
        # reward du step courant
        self.current_reward += self.locals["rewards"][0]

        # fin d’épisode
        if self.locals["dones"][0]:
            self.episode_rewards.append(self.current_reward)
            self.episode_count += 1

            # affichage tous les 10 épisodes
            if self.episode_count % 10 == 0:
                mean_10 = np.mean(self.episode_rewards[-10:])
                print(
                    f"Épisode {self.episode_count} | "
                    f"Reward épisode = {self.current_reward:.2f} | "
                    f"Moyenne (10) = {mean_10:.2f}"
                )

            self.current_reward = 0

        return True


In [8]:
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np
import os
import matplotlib.pyplot as plt


class CleanStatsVideoAndSaveCallback(BaseCallback):
    def __init__(
        self,
        model_ref,
        make_video_env_fn,
        print_every=10,
        save_every=100,
        video_length=1000,
        verbose=0
    ):
        super().__init__(verbose)

        self.model_ref = model_ref
        self.make_video_env_fn = make_video_env_fn

        self.print_every = print_every
        self.save_every = save_every
        self.video_length = video_length

        self.episode_rewards = []

        self.base_dir = "outputs"
        self.video_dir = os.path.join(self.base_dir, "videos")
        self.graph_dir = os.path.join(self.base_dir, "graphs")
        self.model_dir = os.path.join(self.base_dir, "models")

        os.makedirs(self.video_dir, exist_ok=True)
        os.makedirs(self.graph_dir, exist_ok=True)
        os.makedirs(self.model_dir, exist_ok=True)

    def _on_step(self) -> bool:
        for info in self.locals["infos"]:
            if "episode" in info:
                reward = info["episode"]["r"]
                self.episode_rewards.append(reward)
                ep = len(self.episode_rewards)

                if ep % self.print_every == 0:
                    last = self.episode_rewards[-self.print_every:]
                    print(
                        f"[PPO] Episode {ep:4d} | "
                        f"Mean = {np.mean(last):7.1f} | "
                        f"Min = {np.min(last):7.1f} | "
                        f"Max = {np.max(last):7.1f}"
                    )

                if ep % self.save_every == 0:
                    self.save_graph(ep)
                    self.save_model(ep)
                    self.record_video(ep)

        return True

    def save_graph(self, episode):
        plt.figure(figsize=(10, 5))
        plt.plot(self.episode_rewards, linewidth=2)
        plt.xlabel("Episode")
        plt.ylabel("Reward")
        plt.title("PPO Training — Reward per Episode")
        plt.grid(alpha=0.3)

        path = os.path.join(self.graph_dir, f"reward_curve_ep_{episode}.png")
        plt.savefig(path, dpi=150, bbox_inches="tight")
        plt.close()

    def save_model(self, episode):
        path = os.path.join(self.model_dir, f"ppo_model_ep_{episode}")
        self.model_ref.save(path)

    def record_video(self, episode):
        from stable_baselines3.common.vec_env import VecVideoRecorder

        env = self.make_video_env_fn()
        env = VecVideoRecorder(
            env,
            video_folder=self.video_dir,
            record_video_trigger=lambda x: x == 0,
            video_length=self.video_length,
            name_prefix=f"ppo_ep_{episode}"
        )

        obs = env.reset()
        for _ in range(self.video_length):
            action, _ = self.model_ref.predict(obs, deterministic=True)
            obs, _, done, _ = env.step(action)
            if done:
                break

        env.close()
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np
import os
import matplotlib.pyplot as plt


class CleanStatsVideoAndSaveCallback(BaseCallback):
    def __init__(
        self,
        model_ref,
        make_video_env_fn,
        print_every=10,
        save_every=100,
        video_length=1000,
        verbose=0
    ):
        super().__init__(verbose)

        self.model_ref = model_ref
        self.make_video_env_fn = make_video_env_fn

        self.print_every = print_every
        self.save_every = save_every
        self.video_length = video_length

        self.episode_rewards = []

        self.base_dir = "outputs"
        self.video_dir = os.path.join(self.base_dir, "videos")
        self.graph_dir = os.path.join(self.base_dir, "graphs")
        self.model_dir = os.path.join(self.base_dir, "models")

        os.makedirs(self.video_dir, exist_ok=True)
        os.makedirs(self.graph_dir, exist_ok=True)
        os.makedirs(self.model_dir, exist_ok=True)

    def _on_step(self) -> bool:
        for info in self.locals["infos"]:
            if "episode" in info:
                reward = info["episode"]["r"]
                self.episode_rewards.append(reward)
                ep = len(self.episode_rewards)

                if ep % self.print_every == 0:
                    last = self.episode_rewards[-self.print_every:]
                    print(
                        f"[PPO] Episode {ep:4d} | "
                        f"Mean = {np.mean(last):7.1f} | "
                        f"Min = {np.min(last):7.1f} | "
                        f"Max = {np.max(last):7.1f}"
                    )

                if ep % self.save_every == 0:
                    self.save_graph(ep)
                    self.save_model(ep)
                    self.record_video(ep)

        return True

    def save_graph(self, episode):
        plt.figure(figsize=(10, 5))
        plt.plot(self.episode_rewards, linewidth=2)
        plt.xlabel("Episode")
        plt.ylabel("Reward")
        plt.title("PPO Training — Reward per Episode")
        plt.grid(alpha=0.3)

        path = os.path.join(self.graph_dir, f"reward_curve_ep_{episode}.png")
        plt.savefig(path, dpi=150, bbox_inches="tight")
        plt.close()

    def save_model(self, episode):
        path = os.path.join(self.model_dir, f"ppo_model_ep_{episode}")
        self.model_ref.save(path)

    def record_video(self, episode):
        from stable_baselines3.common.vec_env import VecVideoRecorder

        env = self.make_video_env_fn()
        env = VecVideoRecorder(
            env,
            video_folder=self.video_dir,
            record_video_trigger=lambda x: x == 0,
            video_length=self.video_length,
            name_prefix=f"ppo_ep_{episode}"
        )

        obs = env.reset()
        for _ in range(self.video_length):
            action, _ = self.model_ref.predict(obs, deterministic=True)
            obs, _, done, _ = env.step(action)
            if done:
                break

        env.close()


## Envs

In [9]:
import os
import gymnasium as gym

from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage
from stable_baselines3.common.atari_wrappers import WarpFrame


# ============================================================
# Global environment configuration
# ============================================================
ENV_ID = "CarRacing-v3"
GRAY_SCALE = True
N_STACK = 4


# ============================================================
# Wrapper selection
# ============================================================
def _get_wrapper():
    """
    Return the observation wrapper used for preprocessing.
    """
    return WarpFrame if GRAY_SCALE else None


# ============================================================
# Training environment
# ============================================================
def make_train_env(log_dir=None, n_envs=1):
    """
    Create the training environment for PPO.
    """
    wrapper_class = _get_wrapper()

    env = make_vec_env(
        ENV_ID,
        n_envs=n_envs,
        wrapper_class=wrapper_class,
        monitor_dir=log_dir
    )

    env = VecFrameStack(env, n_stack=N_STACK)
    env = VecTransposeImage(env)

    return env


# ============================================================
# Evaluation environment
# ============================================================
def make_eval_env(n_envs=1):
    """
    Environment used for evaluation (no rendering).
    """
    wrapper_class = _get_wrapper()

    env = make_vec_env(
        ENV_ID,
        n_envs=n_envs,
        wrapper_class=wrapper_class
    )

    env = VecFrameStack(env, n_stack=N_STACK)
    env = VecTransposeImage(env)

    return env


# ============================================================
# Video environment
# ============================================================
def make_video_env():
    """
    Environment used for video recording.
    """
    wrapper_class = _get_wrapper()

    env = make_vec_env(
        ENV_ID,
        n_envs=1,
        wrapper_class=wrapper_class
    )

    env = VecFrameStack(env, n_stack=N_STACK)
    env = VecTransposeImage(env)

    return env


# Experiment Configuration


In [10]:

rl_type = "PPO"
env_str = "CarRacing-v3"

log_dir = f"./logs/{env_str}/{rl_type}"
os.makedirs(log_dir, exist_ok=True)

total_timesteps = 1_000_000
eval_freq = 50_000


In [11]:
# =========================
# Environment Creation
# =========================
env = make_train_env(log_dir=log_dir, n_envs=1)
env_val = make_eval_env(n_envs=1)

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.12/dist-packages/

In [12]:


eval_callback = EvalCallback(
    env_val,
    best_model_save_path=log_dir,
    log_path=log_dir,
    eval_freq=eval_freq,
    render=False,
    deterministic=True,
    n_eval_episodes=20
)

checkpoint_callback = CheckpointCallback(
    save_freq=eval_freq,
    save_path=os.path.join(log_dir, "checkpoint")
)


# Entraînement

## Entraînement sur 900 épisodes. Suivi Vidéo

In [ ]:

model = PPO(
    "CnnPolicy",
    env,
    ent_coef=0.0075,
    verbose=0,
)


In [ ]:
callback = CleanStatsVideoAndSaveCallback(
    model_ref=model,
    make_video_env_fn=make_video_env,
    print_every=10,
    save_every=100,
    video_length=1000
)


model.learn(
    total_timesteps=1_000_000,
    callback=callback,
    reset_num_timesteps=False,
    progress_bar=False
)


[PPO] Episode   10 | Mean(last 10) =   -49.3 | Min =   -62.5 | Max =   -39.8
[PPO] Episode   20 | Mean(last 10) =   -30.8 | Min =   -57.4 | Max =    -3.3


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/vec_env/vec_video_recorder.py:157: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")


[PPO] Episode   30 | Mean(last 10) =   -45.2 | Min =   -75.6 | Max =   -17.6
[PPO] Episode   40 | Mean(last 10) =     3.4 | Min =   -72.3 | Max =   168.0
[PPO] Episode   50 | Mean(last 10) =    59.7 | Min =   -31.2 | Max =   212.7
[PPO] Episode   60 | Mean(last 10) =   227.6 | Min =  -125.6 | Max =   480.9
[PPO] Episode   70 | Mean(last 10) =   377.7 | Min =    -2.7 | Max =   560.0
[PPO] Episode   80 | Mean(last 10) =   314.5 | Min =    50.3 | Max =   542.4
[PPO] Episode   90 | Mean(last 10) =   365.1 | Min =   139.0 | Max =   592.0
[PPO] Episode  100 | Mean(last 10) =   401.6 | Min =   -24.5 | Max =   788.1
📈 Graph saved: reward_curve_ep_100.png
💾 Model saved: ppo_model_ep_100.zip
🎬 Recording video at episode 100
Saving video to /content/outputs/videos/ppo_ep_100-step-0-to-step-1000.mp4


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Moviepy - Building video /content/outputs/videos/ppo_ep_100-step-0-to-step-1000.mp4.
Moviepy - Writing video /content/outputs/videos/ppo_ep_100-step-0-to-step-1000.mp4



...

Moviepy - Done !
Moviepy - video ready /content/outputs/videos/ppo_ep_600-step-0-to-step-1000.mp4
✅ Video saved: ppo_ep_600.mp4
[PPO] Episode  610 | Mean(last 10) =   737.3 | Min =   462.5 | Max =   871.7
[PPO] Episode  620 | Mean(last 10) =   626.6 | Min =   373.7 | Max =   864.7
[PPO] Episode  630 | Mean(last 10) =   610.5 | Min =   358.7 | Max =   850.7
[PPO] Episode  640 | Mean(last 10) =   616.9 | Min =    94.5 | Max =   917.3
[PPO] Episode  650 | Mean(last 10) =   281.4 | Min =   -66.4 | Max =   855.6
[PPO] Episode  660 | Mean(last 10) =   787.2 | Min =   442.1 | Max =   893.2
[PPO] Episode  670 | Mean(last 10) =   772.7 | Min =   574.9 | Max =   919.2
[PPO] Episode  680 | Mean(last 10) =   858.6 | Min =   781.8 | Max =   904.7
[PPO] Episode  690 | Mean(last 10) =   867.6 | Min =   763.1 | Max =   914.1
[PPO] Episode  700 | Mean(last 10) =   569.4 | Min =    56.4 | Max =   892.8
📈 Graph saved: reward_curve_ep_700.png
💾 Model saved: ppo_model_ep_700.zip
🎬 Recording video at episod

Moviepy - Done !
Moviepy - video ready /content/outputs/videos/ppo_ep_700-step-0-to-step-1000.mp4
✅ Video saved: ppo_ep_700.mp4
[PPO] Episode  710 | Mean(last 10) =   851.8 | Min =   483.9 | Max =   911.6
[PPO] Episode  720 | Mean(last 10) =   581.7 | Min =    39.3 | Max =   911.0
[PPO] Episode  730 | Mean(last 10) =   699.7 | Min =   245.8 | Max =   915.3
[PPO] Episode  740 | Mean(last 10) =   703.3 | Min =   -30.5 | Max =   914.6
[PPO] Episode  750 | Mean(last 10) =   602.7 | Min =    79.7 | Max =   914.0
[PPO] Episode  760 | Mean(last 10) =   506.7 | Min =   -60.3 | Max =   798.6
[PPO] Episode  770 | Mean(last 10) =   621.9 | Min =    34.3 | Max =   848.6
[PPO] Episode  780 | Mean(last 10) =   752.9 | Min =   481.8 | Max =   873.2
[PPO] Episode  790 | Mean(last 10) =   683.3 | Min =    54.4 | Max =   884.1
[PPO] Episode  800 | Mean(last 10) =   670.5 | Min =    60.7 | Max =   914.8
📈 Graph saved: reward_curve_ep_800.png
💾 Model saved: ppo_model_ep_800.zip
🎬 Recording video at episod

Moviepy - Done !
Moviepy - video ready /content/outputs/videos/ppo_ep_800-step-0-to-step-1000.mp4
✅ Video saved: ppo_ep_800.mp4
[PPO] Episode  810 | Mean(last 10) =   800.4 | Min =   606.6 | Max =   888.8
[PPO] Episode  820 | Mean(last 10) =   782.5 | Min =   261.1 | Max =   910.9
[PPO] Episode  830 | Mean(last 10) =   848.2 | Min =   570.1 | Max =   923.3
[PPO] Episode  840 | Mean(last 10) =   767.6 | Min =    70.6 | Max =   902.0
[PPO] Episode  850 | Mean(last 10) =   704.1 | Min =    58.1 | Max =   917.0
[PPO] Episode  860 | Mean(last 10) =   778.6 | Min =   328.6 | Max =   892.8
[PPO] Episode  870 | Mean(last 10) =   702.6 | Min =   -59.7 | Max =   889.4
[PPO] Episode  880 | Mean(last 10) =   698.1 | Min =   335.5 | Max =   892.7
[PPO] Episode  890 | Mean(last 10) =   772.4 | Min =   266.2 | Max =   921.8
[PPO] Episode  900 | Mean(last 10) =   668.1 | Min =   202.2 | Max =   925.7
📈 Graph saved: reward_curve_ep_900.png
💾 Model saved: ppo_model_ep_900.zip
🎬 Recording video at episod

Moviepy - Done !
Moviepy - video ready /content/outputs/videos/ppo_ep_900-step-0-to-step-1000.mp4
✅ Video saved: ppo_ep_900.mp4


KeyboardInterrupt: 

## Entrainement Best Model Callback

La méthode load permet de recharger l’agent PPO exactement dans l’état où il a été sauvegardé, afin de poursuivre l’entraînement ou d’évaluer sa politique.

In [ ]:

model = PPO.load(
     "/content/ppo_model_ep_900.zip",
     env=env
 )


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:

 best_callback = BestModelCallback(
     model_ref=model,
     save_path="/content/outputs/models/BEST_MODEL_FINAL.zip"
 )


## Tentative de réaintrenement après 900 épisodes.

Malheuresement la session a crash après 7h d'entrainement et j'ai perdu tous mon GPU.

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np
import os
import matplotlib.pyplot as plt


class CleanStatsAndSaveCallback(BaseCallback):
    def __init__(
        self,
        model_ref,
        print_every=20,
        save_every=100,
        verbose=0
    ):
        super().__init__(verbose)

        self.model_ref = model_ref
        self.print_every = print_every
        self.save_every = save_every

        self.episode_rewards = []

        self.base_dir = "outputs"
        self.graph_dir = os.path.join(self.base_dir, "graphs")
        self.model_dir = os.path.join(self.base_dir, "models")

        os.makedirs(self.graph_dir, exist_ok=True)
        os.makedirs(self.model_dir, exist_ok=True)

    def _on_step(self) -> bool:
        for info in self.locals["infos"]:
            if "episode" in info:
                reward = info["episode"]["r"]
                self.episode_rewards.append(reward)
                ep = len(self.episode_rewards)

                # ===== affichage stats =====
                if ep % self.print_every == 0:
                    last = self.episode_rewards[-self.print_every:]
                    print(
                        f"[PPO] Episode {ep:4d} | "
                        f"Mean = {np.mean(last):7.1f} | "
                        f"Min = {np.min(last):7.1f} | "
                        f"Max = {np.max(last):7.1f}"
                    )

                # ===== sauvegardes =====
                if ep % self.save_every == 0:
                    self.save_graph(ep)
                    self.save_model(ep)

        return True

    def save_graph(self, episode):
        plt.figure(figsize=(10, 5))
        plt.plot(self.episode_rewards, linewidth=2)
        plt.xlabel("Episode")
        plt.ylabel("Reward")
        plt.title("PPO Training — Reward per Episode")
        plt.grid(alpha=0.3)

        path = os.path.join(self.graph_dir, f"reward_curve_ep_{episode}.png")
        plt.savefig(path, dpi=150, bbox_inches="tight")
        plt.close()

    def save_model(self, episode):
        path = os.path.join(self.model_dir, f"ppo_model_ep_{episode}")
        self.model_ref.save(path)


In [ ]:
from stable_baselines3.common.callbacks import CallbackList

best_callback = BestModelCallback(
    model_ref=model,
    save_path="outputs/models/new"
)

stats_callback = CleanStatsAndSaveCallback(
    model_ref=model,
    print_every=20,
    save_every=100
)

callbacks = CallbackList([
    best_callback,
    stats_callback
])

model.learn(
    total_timesteps=2_000_000,
    callback=callbacks,
    reset_num_timesteps=False,
    progress_bar=False
)


🏆 New BEST model saved | Reward = 11.86
🏆 New BEST model saved | Reward = 874.82
🏆 New BEST model saved | Reward = 875.52
🏆 New BEST model saved | Reward = 904.20
🏆 New BEST model saved | Reward = 911.60
[PPO] Episode   20 | Mean =   562.8 | Min =    11.9 | Max =   911.6
🏆 New BEST model saved | Reward = 918.60
🏆 New BEST model saved | Reward = 922.40
🏆 New BEST model saved | Reward = 923.00
[PPO] Episode   40 | Mean =   635.8 | Min =   -11.6 | Max =   923.0
[PPO] Episode   60 | Mean =   491.5 | Min =   -32.9 | Max =   888.1
[PPO] Episode   80 | Mean =   597.0 | Min =    29.3 | Max =   913.3
[PPO] Episode  100 | Mean =   648.1 | Min =    51.6 | Max =   921.7
[PPO] Episode  120 | Mean =   718.4 | Min =   278.3 | Max =   909.0
🏆 New BEST model saved | Reward = 923.30
[PPO] Episode  140 | Mean =   632.8 | Min =    83.2 | Max =   923.3
[PPO] Episode  160 | Mean =   456.5 | Min =    34.8 | Max =   892.3
[PPO] Episode  180 | Mean =   315.3 | Min =   -27.8 | Max =   772.7
[PPO] Episode  200 |

## Malheuresement je n'ai pas pu enregistré le modèle final...

## Générer une vidéo finale du meilleur modèle.

In [15]:

# ================================
# FINAL BEST MODEL VIDEO (30s)
# ================================

# Pour tester mes résultats charger le modèle sur Colllab et executer les cellules
#BEST_MODEL_PATH = "/content/outputs/models/BEST_MODEL_FINAL.zip"
BEST_MODEL_PATH = "BEST_MODEL_FINAL.zip"
VIDEO_DIR = "/content/outputs/videos"
VIDEO_NAME = "BEST_MODEL_FINAL"

model = PPO.load(
    BEST_MODEL_PATH,
    env=make_video_env()
)

env = make_video_env()
env = VecVideoRecorder(
    env,
    video_folder=VIDEO_DIR,
    record_video_trigger=lambda x: True,
    video_length=900,
    name_prefix=VIDEO_NAME
)

obs = env.reset()
for _ in range(900):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, done, _ = env.step(action)
    if done:
        break

env.close()

video_path = os.path.join(
    VIDEO_DIR,
    f"{VIDEO_NAME}-step-0-to-step-900.mp4"
)
print(" Final video generated:", video_path)

files.download(video_path)



KeyboardInterrupt: 

# FINAL EVALUATION

In [13]:

from stable_baselines3.common.evaluation import evaluate_policy
import os

# ================================
# LOAD BEST MODEL (OBJECTIVE)
# ================================

# Version locale (par défaut)
BEST_MODEL_PATH = "BEST_MODEL_FINAL.zip"

# Version Colab
# BEST_MODEL_PATH = "/content/outputs/models/BEST_MODEL_FINAL.zip"

model = PPO.load(
    BEST_MODEL_PATH,
    env=env_val
)

# ================================
# FINAL EVALUATION
# ================================

mean_reward, std_reward = evaluate_policy(
    model,
    env_val,
    n_eval_episodes=20,
    deterministic=True
)

print(" Final BEST Model Evaluation")
print(f"Mean reward: {mean_reward:.2f}")
print(f"Std reward : {std_reward:.2f}")

# ================================
# SAVE FINAL MODEL (CLEAN NAME)
# ================================

FINAL_MODEL_PATH = os.path.join(log_dir, "final_model_800plus")
model.save(FINAL_MODEL_PATH)

print("Final model saved to:", FINAL_MODEL_PATH + ".zip")

# ================================
# CLEANUP
# ================================

env.close()
env_val.close()


 Final BEST Model Evaluation
Mean reward: 831.42
Std reward : 165.20
Final model saved to: ./logs/CarRacing-v3/PPO/final_model_800plus.zip


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
